In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [1]:
#traditional ELM on MNIST
import numpy as np
from tensorflow.keras.datasets import mnist
from sklearn.preprocessing import LabelBinarizer
from sklearn.metrics import accuracy_score

class ELM:
    def __init__(self, input_dim, hidden_dim, activation='sigmoid'):
        self.input_dim = input_dim
        self.hidden_dim = hidden_dim
        self.activation = activation
        self.W = np.random.randn(self.hidden_dim, self.input_dim)
        self.b = np.random.randn(self.hidden_dim)

    def _activation(self, x):
        if self.activation == 'sigmoid':
            return 1 / (1 + np.exp(-x))
        elif self.activation == 'tanh':
            return np.tanh(x)
        else:
            raise ValueError("Unsupported activation function.")

    def fit(self, X, y):
        H = self._activation(np.dot(X, self.W.T) + self.b)
        self.beta = np.dot(np.linalg.pinv(H), y)

    def predict(self, X):
        H = self._activation(np.dot(X, self.W.T) + self.b)
        y_pred = np.dot(H, self.beta)
        return y_pred

# Load MNIST from keras (with predefined train/test split)
print("Loading MNIST dataset...")
(X_train, y_train), (X_test, y_test) = mnist.load_data()

# Preprocessing
X_train = X_train.reshape(-1, 28*28).astype(np.float32) / 255.0
X_test = X_test.reshape(-1, 28*28).astype(np.float32) / 255.0

# One-hot encode labels
encoder = LabelBinarizer()
Y_train = encoder.fit_transform(y_train)
Y_test = encoder.transform(y_test)

# Initialize and train ELM
print("Training ELM...")
elm = ELM(input_dim=784, hidden_dim=1000, activation='sigmoid')
elm.fit(X_train, Y_train)

# Predict and evaluate
print("Evaluating...")
y_pred_probs = elm.predict(X_test)
y_pred = np.argmax(y_pred_probs, axis=1)

acc = accuracy_score(y_test, y_pred)
print(f"Test Accuracy: {acc * 100:.2f}%")


2025-06-26 21:57:23.673814: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1750975043.936886      35 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1750975044.006525      35 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


Loading MNIST dataset...
11490434/11490434 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
Training ELM...
Evaluating...
Test Accuracy: 93.27%


In [2]:
#traditional ELM on fashion-MNIST
import numpy as np
from tensorflow.keras.datasets import fashion_mnist
from sklearn.preprocessing import LabelBinarizer
from sklearn.metrics import accuracy_score

class ELM:
    def __init__(self, input_dim, hidden_dim, activation='sigmoid'):
        self.input_dim = input_dim
        self.hidden_dim = hidden_dim
        self.activation = activation
        self.W = np.random.randn(self.hidden_dim, self.input_dim)
        self.b = np.random.randn(self.hidden_dim)

    def _activation(self, x):
        if self.activation == 'sigmoid':
            return 1 / (1 + np.exp(-x))
        elif self.activation == 'tanh':
            return np.tanh(x)
        else:
            raise ValueError("Unsupported activation")

    def fit(self, X, y):
        H = self._activation(np.dot(X, self.W.T) + self.b)
        self.beta = np.dot(np.linalg.pinv(H), y)

    def predict(self, X):
        H = self._activation(np.dot(X, self.W.T) + self.b)
        return np.dot(H, self.beta)

# Load Fashion-MNIST
print("Loading Fashion-MNIST...")
(X_train, y_train), (X_test, y_test) = fashion_mnist.load_data()
X_train = X_train.reshape(-1, 28*28).astype(np.float32) / 255.0
X_test = X_test.reshape(-1, 28*28).astype(np.float32) / 255.0

# One-hot encode
encoder = LabelBinarizer()
Y_train = encoder.fit_transform(y_train)
Y_test = encoder.transform(y_test)

# Train ELM
elm = ELM(input_dim=784, hidden_dim=1000, activation='sigmoid')
elm.fit(X_train, Y_train)

# Evaluate
y_pred_probs = elm.predict(X_test)
y_pred = np.argmax(y_pred_probs, axis=1)
acc = accuracy_score(y_test, y_pred)
print(f"Fashion-MNIST Accuracy: {acc * 100:.2f}%")


Loading Fashion-MNIST...
29515/29515 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
26421880/26421880 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
5148/5148 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
4422102/4422102 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
Fashion-MNIST Accuracy: 84.42%


In [5]:
#traditional ELM on K-MNIST

import numpy as np
import tensorflow_datasets as tfds

def load_kmnist_tfds():
    ds_train, ds_test = tfds.load('kmnist', split=['train', 'test'], as_supervised=True, batch_size=-1)
    x_tr, y_tr = tfds.as_numpy(ds_train)
    x_te, y_te = tfds.as_numpy(ds_test)

    # Reshape and normalize
    x_tr = x_tr.reshape(-1, 28 * 28).astype(np.float32).T / 255.0  # shape: (784, N)
    x_te = x_te.reshape(-1, 28 * 28).astype(np.float32).T / 255.0
    return x_tr, y_tr, x_te, y_te

class ELM_CPU:
    def __init__(self, input_dim, hidden_dim, activation='sigmoid'):
        self.input_dim = input_dim
        self.hidden_dim = hidden_dim
        self.activation = activation
        self.W = np.random.randn(hidden_dim, input_dim)
        self.b = np.random.randn(hidden_dim, 1)

    def _activation(self, x):
        if self.activation == 'sigmoid':
            return 1 / (1 + np.exp(-x))
        elif self.activation == 'tanh':
            return np.tanh(x)
        else:
            raise ValueError("Unsupported activation function.")

    def fit(self, X, Y):
        H = self._activation(self.W @ X + self.b)  # H shape: (hidden_dim, N)
        self.beta = np.linalg.pinv(H.T) @ Y.T      # beta shape: (hidden_dim, 10)

    def predict(self, X):
        H = self._activation(self.W @ X + self.b)
        Y_pred = self.beta.T @ H                   # shape: (10, N)
        return np.argmax(Y_pred, axis=0)

# Load and preprocess KMNIST data
x_tr, y_tr, x_te, y_te = load_kmnist_tfds()
Y_train = (np.eye(10)[y_tr].T * 2) - 1            # shape: (10, N), values in {-1, 1}

# Initialize and train ELM
elm = ELM_CPU(input_dim=784, hidden_dim=1000, activation='sigmoid')
elm.fit(x_tr, Y_train)

# Predict
y_pred = elm.predict(x_te)

# Evaluate
accuracy = np.mean(y_pred == y_te)
print(f"KMNIST Accuracy (CPU only): {accuracy * 100:.2f}%")


KMNIST Accuracy (CPU only): 70.07%
